In [15]:
import pandas as pd
from sqlalchemy import create_engine, text
import urllib.parse

In [ ]:
# Load and clean data
match_df = pd.read_csv(r"D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary\summary_cleaned.csv")
match_df.drop(columns=["Unnamed: 0"], inplace=True)

In [17]:
match_df.columns

Index(['match_id', 'season', 'match_date', 'event_name', 'gender',
       'match_type', 'team1', 'team2', 'toss_winner', 'toss_decision',
       'winner', 'ground'],
      dtype='object')

In [18]:
match_df.isnull().sum()

match_id         0
season           0
match_date       0
event_name       0
gender           0
match_type       0
team1            0
team2            0
toss_winner      0
toss_decision    0
winner           0
ground           0
dtype: int64

In [19]:
match_df["event_name"] = match_df["event_name"].fillna("NA")

In [ ]:
# Make sure match_date is a datetime object
match_df['match_date'] = pd.to_datetime(match_df['match_date'], errors='coerce')

In [ ]:
# Validate match_id
assert match_df['match_id'].is_unique, "match_id has duplicates!"
assert not match_df['match_id'].isnull().any(), "match_id has nulls!"
match_df['match_id'] = match_df['match_id'].astype(int)

In [13]:
from sqlalchemy import create_engine, text
import urllib.parse
import pandas as pd

POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "cricket_data",
}

# Optional: Encode the password in case it contains special characters
password = urllib.parse.quote(POSTGRES_CREDENTIALS["password"])


In [20]:
# Create SQLAlchemy engine for PostgreSQL
engine = create_engine(
    f"postgresql+psycopg2://{POSTGRES_CREDENTIALS['username']}:{password}@{POSTGRES_CREDENTIALS['host']}:{POSTGRES_CREDENTIALS['port']}/{POSTGRES_CREDENTIALS['database']}"
)

# Table name
table_name = 'matches'

# SQL to create table if it does not exist
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    match_id      INTEGER PRIMARY KEY,
    season        VARCHAR(10),
    match_date    DATE,
    event_name    VARCHAR(100),
    gender        VARCHAR(10),
    match_type    VARCHAR(20),
    team1         VARCHAR(50),
    team2         VARCHAR(50),
    toss_winner   VARCHAR(50),
    toss_decision VARCHAR(10),
    winner        VARCHAR(50),
    ground        VARCHAR(100)
);
"""

# Run everything
try:
    with engine.connect() as conn:
        # Get existing match_ids
        existing_ids = pd.read_sql(f"SELECT match_id FROM {table_name}", conn)
        existing_ids_set = set(existing_ids['match_id'])

    # Filter new matches only
    new_matches_df = match_df[~match_df['match_id'].isin(existing_ids_set)]

    if not new_matches_df.empty:
        new_matches_df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
        print(f"{len(new_matches_df)} new match(es) inserted.")
    else:
        print("No new matches to insert.")

except Exception as e:
    print(f"An error occurred: {e}")


No new matches to insert.


In [21]:
import pandas as pd
import os

# Path to the full match summary CSV
input_file = r"D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary\summary_cleaned.csv"

# Load the data
match_df = pd.read_csv(input_file)

# Fill missing gender/match_type just in case
match_df['gender'] = match_df['gender'].fillna('unknown')
match_df['match_type'] = match_df['match_type'].fillna('unknown')

# Base output directory
base_output_dir = r"D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary"

# Group by gender and match_type
for (gender, match_type), group_df in match_df.groupby(['gender', 'match_type']):
    gender_dir = os.path.join(base_output_dir, gender.lower())
    os.makedirs(gender_dir, exist_ok=True)  # Create the folder if it doesn't exist
    
    filename = f"{match_type.lower()}_{gender.lower()}_match_summary_cleaned.csv"
    output_path = os.path.join(gender_dir, filename)

    group_df.to_csv(output_path, index=False)
    print(f"Saved: {output_path}")


Saved: D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary\female\it20_female_match_summary_cleaned.csv
Saved: D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary\female\odi_female_match_summary_cleaned.csv
Saved: D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary\female\odm_female_match_summary_cleaned.csv
Saved: D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary\female\t20_female_match_summary_cleaned.csv
Saved: D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary\female\test_female_match_summary_cleaned.csv
Saved: D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary\male\it20_male_match_summary_cleaned.csv
Saved: D:\GITHUB\Predictive-Analytics-for-Crick